# 📝 Exercise M3.01

The goal is to write an exhaustive search to find the best parameters
combination maximizing the model generalization performance.

Here we use a small subset of the Adult Census dataset to make the code faster
to execute. Once your code works on the small subset, try to change
`train_size` to a larger value (e.g. 0.8 for 80% instead of 20%).

In [2]:
import pandas as pd

from sklearn.model_selection import train_test_split

adult_census = pd.read_csv(r"..\data\adult-census.csv")

target_name = "class"
target = adult_census[target_name]
data = adult_census.drop(columns=[target_name, "education-num"])

data_train, data_test, target_train, target_test = train_test_split(
    data, target, train_size=0.2, random_state=42
)

In [3]:
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import OrdinalEncoder

categorical_preprocessor = OrdinalEncoder(
    handle_unknown="use_encoded_value", unknown_value=-1
)
preprocessor = make_column_transformer(
    (categorical_preprocessor, selector(dtype_include=object)),
    remainder="passthrough",
)

from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.pipeline import Pipeline

model = Pipeline(
    [
        ("preprocessor", preprocessor),
        ("classifier", HistGradientBoostingClassifier(random_state=42)),
    ]
)

Use the previously defined model (called `model`) and using two nested `for`
loops, make a search of the best combinations of the `learning_rate` and
`max_leaf_nodes` parameters. In this regard, you need to train and test the
model by setting the parameters. The evaluation of the model should be
performed using `cross_val_score` on the training set. Use the following
parameters search:
- `learning_rate` for the values 0.01, 0.1, 1 and 10. This parameter controls
  the ability of a new tree to correct the error of the previous sequence of
  trees
- `max_leaf_nodes` for the values 3, 10, 30. This parameter controls the depth
  of each tree.

In [4]:
from sklearn.model_selection import cross_val_score

learning_rate = [0.01, 0.1, 1, 10]
max_leaf_nodes = [3, 10, 30]
best_score = 0
best_params = {}

for lr in learning_rate:
    for mln in max_leaf_nodes:
        print(( f"Evaluate the model with learning rate {lr:.3f} , and max leaf nodes {mln}" ),end="")    
        
        model.set_params(classifier__learning_rate=lr, classifier__max_leaf_nodes=mln)
            
        scores = cross_val_score(model, data_train, target_train, cv=2)
        mean_score = scores.mean()
        print(f"score: {mean_score:.3f}")
        if mean_score > best_score:
            best_score = mean_score
            best_params = {"learning_rate": lr, "max_leaf_nodes": mln}
            print(f"Found new best model with score: {best_score:.3f}")

print(f"The best accuracy achieved is {best_score:.3f}  , and The best parameters found are: {best_params}")

Evaluate the model with learning rate 0.010 , and max leaf nodes 3score: 0.789
Found new best model with score: 0.789
Evaluate the model with learning rate 0.010 , and max leaf nodes 10score: 0.813
Found new best model with score: 0.813
Evaluate the model with learning rate 0.010 , and max leaf nodes 30score: 0.842
Found new best model with score: 0.842
Evaluate the model with learning rate 0.100 , and max leaf nodes 3score: 0.847
Found new best model with score: 0.847
Evaluate the model with learning rate 0.100 , and max leaf nodes 10score: 0.859
Found new best model with score: 0.859
Evaluate the model with learning rate 0.100 , and max leaf nodes 30score: 0.857
Evaluate the model with learning rate 1.000 , and max leaf nodes 3score: 0.852
Evaluate the model with learning rate 1.000 , and max leaf nodes 10score: 0.833
Evaluate the model with learning rate 1.000 , and max leaf nodes 30score: 0.828
Evaluate the model with learning rate 10.000 , and max leaf nodes 3score: 0.288
Evaluate

Now use the test set to score the model using the best parameters that we
found using cross-validation. You will have to refit the model over the full
training set.

In [8]:
best_learning_rate = best_params["learning_rate"]
best_max_leaf_nodes = best_params["max_leaf_nodes"]

model.set_params(classifier__learning_rate=best_learning_rate, classifier__max_leaf_nodes=best_max_leaf_nodes)
model.fit(data_train, target_train)
test_score = model.score(data_test, target_test)

print(f"Test score of the model: {test_score:.2f}")

Test score of the model: 0.87
